In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from utils.orventro import deviation_metric_optimized, dev_metric
from datetime import datetime

xgb.set_config(verbosity=0)

In [30]:
train_df = pd.read_csv('./data/train.csv', index_col='id')
target = 'per_square_meter_price'

In [31]:
def prep(df):
    df = df.drop(['city', 'osm_city_nearest_name', 'date'], axis=1)
    df['floor'] = np.where(pd.to_numeric(df['floor'], errors='coerce').isnull().to_numpy(), 
                           -1, 
                           df['floor'].to_numpy())
    df['floor'] = pd.to_numeric(df['floor'], errors='coerce')
    regions = df['region'].unique()
    reg_dict = dict(zip(regions, range(len(regions))))
    print(reg_dict)
    df['region'] = [reg_dict[r] for r in df['region']]
    df['street'] = [s if type(s) == float else float(s[1:]) for s in df['street']]
    return df.astype(float)

In [32]:
train_df = prep(train_df)
# valid = train_df.iloc[100000:]
train = train_df

{'Пермский край': 0, 'Московская область': 1, 'Ярославская область': 2, 'Кемеровская область': 3, 'Москва': 4, 'Самарская область': 5, 'Татарстан': 6, 'Саратовская область': 7, 'Краснодарский край': 8, 'Свердловская область': 9, 'Брянская область': 10, 'Башкортостан': 11, 'Калининградская область': 12, 'Челябинская область': 13, 'Воронежская область': 14, 'Томская область': 15, 'Санкт-Петербург': 16, 'Ростовская область': 17, 'Орловская область': 18, 'Тульская область': 19, 'Белгородская область': 20, 'Нижегородская область': 21, 'Вологодская область': 22, 'Мордовия': 23, 'Омская область': 24, 'Алтайский край': 25, 'Иркутская область': 26, 'Удмуртия': 27, 'Ставропольский край': 28, 'Ханты-Мансийский АО': 29, 'Пензенская область': 30, 'Тюменская область': 31, 'Ленинградская область': 32, 'Ульяновская область': 33, 'Калужская область': 34, 'Красноярский край': 35, 'Смоленская область': 36, 'Костромская область': 37, 'Волгоградская область': 38, 'Кировская область': 39, 'Курская область':

In [5]:
def test_model1(model, train, valid):
    model.fit(train.drop([target], axis=1), train[target])
    pred = model.predict(valid.drop([target], axis=1))
    err = deviation_metric_optimized(pred, valid[target].to_numpy())
    print(model, err)

In [21]:
def train_stack(train):
    weights = train['price_type'].to_numpy() * 99 + 1
#     model_cat = CatBoostRegressor(iterations=100, verbose=0)
#                                  loss_function=deviation_metric_optimized,
#                                  eval_metric=deviation_metric_optimized)
    model_xgb = XGBRegressor(device='GPU')#, obj=dev_metric)
    model_lgb = LGBMRegressor(device='GPU')#, objective=dev_metric)
    level0 = list()
#     level0.append(('cat', model_cat))
    level0.append(('xgb', model_xgb))
    level0.append(('lgb', model_lgb))

    model_logreg = LinearRegression()

    model = StackingRegressor(estimators=level0, 
                              final_estimator=model_logreg, 
                              cv=5)
    model.fit(train.drop([target], axis=1), 
              train[target],
              sample_weight=weights)
    return model

def validate_stack(stack, valid):
    pred = stack.predict(valid.drop([target], axis=1))
    weights = valid['price_type'] * 99 + 1
    return deviation_metric_optimized(valid[target], pred, weights)

def predict(stack, save):
    test_df = pd.read_csv('./data/test.csv', index_col='id')
    test_df = prep(test_df)
    
    pred = pd.DataFrame(index=test_df.index)
    pred.insert(0, target, stack.predict(test_df)/8) #WILL REMOVE
    if save:
        pred.to_csv(f'./out_{datetime.now().time()}.csv')
    
    out_df = pd.read_csv('./out.csv', index_col='id')
    print(deviation_metric_optimized(pred[target], out_df[target]))

In [11]:
stack = train_stack(train)

In [12]:
# err = validate_stack(stack, valid)
# print(err)

In [22]:
predict(stack, False)

3.6816240454948934


In [ ]:
xgb.set_config(verbosity=1)
model_xgb = XGBRegressor(device='GPU')#, obj=dev_metric)
test_model1(model_xgb, train, valid)